In [21]:
import numpy as np
import tensorflow as tf

!nvidia-smi

MODEL_PATH = "../input/cnn-model-benchmarking-models/MobileNetV3_small_ImageNet_62_84_92MF/MobileNetV3_small_ImageNet_62_84_92MF"
DATASET_PATH = "../input/imagenet-val-100/ImageNet_val_100.csv"
DATASET_LABELS_PATH = "../input/imagenet-val-100/ImageNet_val_100_labels.csv"

#Load the Dataset
x_test = np.loadtxt(DATASET_PATH, dtype=int, delimiter=",")
y_test = np.loadtxt(DATASET_LABELS_PATH, dtype=int, delimiter=",")
print(MODEL_PATH)
#Do the preprocessing that is necessary per model. 
if "ResNet50" in MODEL_PATH:
    x_test = x_test.reshape(-1,224,224,3)
    x_test = tf.keras.applications.resnet50.preprocess_input(x_test)
elif "NASNet_large" in MODEL_PATH:
    x_test = x_test.reshape(-1,224,224,3)
    x_test = tf.image.resize(x_test, (331, 331))
    x_test = tf.keras.applications.nasnet.preprocess_input(x_test)
elif "MobileNetV3" in MODEL_PATH:
    x_test = x_test.reshape(-1,224,224,3)
    x_test = tf.cast(x=x_test, dtype=tf.float32)/127.5 - 1.0
elif "LeNet5" in MODEL_PATH:
    x_test = x_test.reshape(-1,32,32,3)
    x_test = tf.image.rgb_to_grayscale(x_test)
    x_test = tf.cast(x=x_test, dtype=tf.float32)/255.0
# Load the model
model = tf.keras.models.load_model(filepath=MODEL_PATH)

print(model.layers[0].input_shape)
print(model.layers[0].dtype)

print(model.layers[-1].output_shape)
print(model.layers[-1].dtype)

from timeit import default_timer as timer
print(x_test.shape)
iterations = x_test.shape[0]
#Transferring inputs in one go
total_time=0.0
acc = 0
start = timer()
output_data = model.predict(x=x_test, batch_size=1)
end = timer()
total_time += end - start # Time in seconds, e.g. 5.38091952400282
for i in range(iterations):
    if(np.argmax(output_data[i]) == y_test[i]):
        acc += 1
print("Transferring %d inputs in one go" %x_test.shape[0])
avg_time = total_time/ iterations
acc = acc / iterations
print("Average time: %.3f ms" %(avg_time*1000))
print("Top1 Accuracy: %.4f" %acc)

#Transfering inputs one by one
total_time=0.0
acc = 0
for i in range(iterations):
    start = timer()
    output_data = model.predict(x=x_test[np.newaxis, i, :])
    end = timer()
    total_time += end - start # Time in seconds, e.g. 5.38091952400282
    if(np.argmax(output_data) == y_test[i]):
        acc += 1
print("Transferring %d inputs one by one" %x_test.shape[0])
avg_time = total_time/ iterations
acc = acc / iterations
print("Average time: %.3f ms" %(avg_time*1000))
print("Top1 Accuracy: %.4f" %acc)